In [52]:
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv
import hvplot.pandas
import param
import random
from datetime import datetime
pn.extension()
# 数据处理
data = pd.read_csv('world_wide_COVID_data_daily_data.csv')
# 转换时间格式
data.loc[:, 'dateRep'] = data['dateRep'].apply(lambda x: datetime.strptime(x,"%d/%m/%Y"))
# 统计洲对应的国家数据
countriesData = {}
for row in data.itertuples():
    if row.continentExp not in countriesData:
        countriesData[row.continentExp] = [row.countriesAndTerritories]
    elif row.countriesAndTerritories not in countriesData[row.continentExp]:
        countriesData[row.continentExp].append(row.countriesAndTerritories)
        
     
class MyPanel(param.Parameterized):
    continents = param.ListSelector(default=['Asia'], objects=countriesData.keys())
    countries = param.ListSelector(default=['China'], objects=countriesData['Asia'])
    months = param.Range(default=(2, 3), bounds=(1, 12))
    dataType  = param.Selector(default="cases", objects=["cases", "deaths"])
    
    _countries = countriesData
    
    @param.depends('continents', watch=True)
    def _update_countries(self):
        countries = []
        for i in self.continents:
            countries= countries + self._countries[i]
        self.param['countries'].objects = countries
        self.countries = [countries[0]]

    @param.depends('countries','months','dataType')
    def view(self):
        start = self.months[0]
        end = self.months[1]
        graph_data = data[(data['continentExp'].isin(self.continents))&(data['countriesAndTerritories'].isin(self.countries))&(data['month']<=end)&(data['month']>=start)]
        timeline = graph_data.hvplot(x='dateRep', y=self.dataType, xlabel='Date', rot=90, sort_date=True, by='countriesAndTerritories')
        table = graph_data.hvplot.table(sortable=True)
        return pn.Column(timeline,table)
        
viewer = MyPanel(name='Setting')
pn.Row(pn.Param(viewer.param, widgets={'dataType': pn.widgets.RadioButtonGroup}), viewer.view)




Row
    [0] Param(MyPanel, name='Setting', widgets={'dataType': <...})
    [1] ParamMethod(method)